# Face detection and recognition training pipeline

The following example illustrates how to fine-tune an InceptionResnetV1 model on your own dataset. This will mostly follow standard pytorch training patterns.

In [1]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

d:\mamba\envs\default\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


#### Define run parameters

The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify `data_dir` to the location of the dataset on wish to finetune on.

In [10]:
data_dir = '../hard_images_copy'

batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 4 # 8

#### Determine if an nvidia GPU is available

In [2]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


#### Define MTCNN module

See `help(MTCNN)` for more details.

In [8]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and obtain cropped faces.

In [4]:
%load_ext autoreload
%autoreload 2

In [11]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.idx_to_class = {i:c for c, i in dataset.class_to_idx.items()}
dataset.idx_to_class

{0: 'Ayumi_Hamasaki',
 1: 'HK_Source',
 2: 'Haruka_Imai',
 3: 'Haruna_Kawaguchi',
 4: 'Other_Source',
 5: 'Yui_Aragaki'}

In [7]:
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]

# loader = DataLoader(
#     dataset,
#     num_workers=workers,
#     batch_size=batch_size,
#     collate_fn=training.collate_pil
# )
loader = DataLoader(dataset, collate_fn=training.collate_pil, num_workers=workers)

# x: image object
# y: save path of corresponding object
for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    
# Remove mtcnn to reduce GPU memory usage
del mtcnn

Batch 1475 of 1475

#### Define Inception Resnet V1 module

See `help(InceptionResnetV1)` for more details.

In [12]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

#### Define optimizer, scheduler, dataset, and dataloader

In [13]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
# dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
# img_inds = np.arange(len(dataset))
# np.random.shuffle(img_inds)
# train_inds = img_inds[:int(0.8 * len(img_inds))]
# val_inds = img_inds[int(0.8 * len(img_inds)):]

dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
path = data_dir + '_cropped'
hk_source_length = len(os.listdir(path + '\\AA_HK_Source'))
hk_source_inds = np.arange(hk_source_length)
other_source_length = len(os.listdir(path + '\\AA_Other_Source'))
other_source_inds = np.arange(hk_source_length, hk_source_length + other_source_length)
img_inds = np.arange(len(dataset))
train_inds = img_inds[:other_source_inds[-1]]
val_inds = img_inds[other_source_inds[-1]:]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

#### Define loss and evaluation functions

In [14]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

#### Train model

In [15]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    if epoch % 5 == 0:
        resnet.eval()
        training.pass_epoch(
            resnet, loss_fn, val_loader,
            batch_metrics=metrics, show_running=True, device=device,
            writer=writer
        )

writer.close()



Initial
----------
Valid |    46/46   | loss:    2.1268 | fps:  101.5943 | acc:    0.0989   

Epoch 1/8
----------
Train |     1/1    | loss:    1.6708 | fps:    8.8665 | acc:    0.4000   
Valid |    46/46   | loss:    2.4793 | fps:  325.5817 | acc:    0.0135   

Epoch 2/8
----------
Train |     1/1    | loss:    0.5420 | fps:   97.0794 | acc:    0.9000   

Epoch 3/8
----------
Train |     1/1    | loss:    0.1755 | fps:   81.9703 | acc:    1.0000   

Epoch 4/8
----------
Train |     1/1    | loss:    0.0220 | fps:   84.0357 | acc:    1.0000   

Epoch 5/8
----------
Train |     1/1    | loss:    0.0086 | fps:   82.6454 | acc:    1.0000   

Epoch 6/8
----------
Train |     1/1    | loss:    0.0066 | fps:   80.6403 | acc:    1.0000   
Valid |    46/46   | loss:    7.0177 | fps:  336.6342 | acc:    0.0082   

Epoch 7/8
----------
Train |     1/1    | loss:    0.0062 | fps:  101.0074 | acc:    1.0000   

Epoch 8/8
----------
Train |     1/1    | loss:    0.0077 | fps:   84.0389 | acc:   